In [ ]:
!pip install webdriver_manager

In [ ]:
!pip install selenium

In [ ]:
!ls /usr/lib/chromium-browser/chromedriver

!apt-get update

!apt-get install -y chromium-browser

!apt-get install chromium-chromedriver

!pip install --upgrade selenium

ls: cannot access '/usr/lib/chromium-browser/chromedriver': No such file or directory
Get:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:4 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [713 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:12 http://security.ubuntu.

In [ ]:
from selenium.webdriver.chrome.options import Options
import pandas as pd
from time import sleep
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
#import undetected_chromedriver as uc
from selenium import webdriver



restaurant_name = 'panchos_burritos'
url ='https://www.yelp.com/biz/panchos-burritos-new-milford?osq=Restaurants'

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('--no-sandbox')
options.add_argument('--headless')
options.add_argument('--disable-gpu')
options.add_argument('--disable-dev-shm-uage')

#driver = webdriver.Chrome(executable_path='/usr/lib/chromium-browser/chromedriver', options=options)
driver = webdriver.Chrome(options=options)

driver.get(url)

In [48]:
data=[]

sleep(2)


while True:
    reviews = driver.find_elements(By.XPATH,' //ul//li[@class=" css-1q2nwpv"]')
    for r in reviews[2:]:
        try: name = r.find_element(By.XPATH,'.//a[@class="css-19v1rkv"]').text
        except: name = ''
        try: location =  r.find_element(By.XPATH,'.//span[@class=" css-qgunke"]').text
        except: location = ''
        try:  date = r.find_element(By.XPATH,'.//span[@class=" css-chan6m"]').text
        except: date = ''
        try: review_text = r.find_element(By.XPATH,'.//span[@class=" raw__09f24__T4Ezm"]').text
        except: review_text= ''

        aaa = r.find_elements(By.XPATH,'.//div[@class="css-14g69b3"]')
        for a in aaa:
            rating = a.get_attribute('aria-label').split(' ')[0].strip()

            data.append([name, rating, location, date, review_text])
        df = pd.DataFrame(data, columns=['name', 'rating', 'location', 'date', 'body'])

    try:
        try: next_page = driver.find_element(By.XPATH,'//a[@class="next-link navigation-button__09f24__m9qRz css-ahgoya"]').click();sleep(3)
        except:
            try:
                next_page = driver.find_element(By.XPATH,'//a[@class="next-link navigation-button__09f24__m9qRz css-ahgoya"]').click();sleep(3)
            except: break
    except: break

In [49]:
df

,name,rating,location,date,body
0,Sarah P.,3,"Montclair, NJ","Mar 7, 2023",Let's start with the good - the ambiance and s...
1,Joseph F.,4,"New Milford, NJ","Dec 23, 2023",Pancho's food and service are very good BUT ri...
2,Dena H.,3,"Englewood, NJ","Jan 26, 2023",I really loved the vibes of this place and the...
3,Ellie O.,4,"Westwood, NJ","Jan 12, 2024",We have way too much fun when we come here! Fo...
4,Lenny G.,3,"Verona, NJ","Jan 14, 2023",The food is pretty mediocre and is way overpri...
...,...,...,...,...,...
633,Janis H.,2,"New Milford, NJ","Jun 16, 2010","This place is great for drinks, guac and chips..."
634,Rita S.,5,"Manhattan, NY","Mar 4, 2012",The bean and cheese burritos are the best I've...
635,C B.,5,"Wood-Ridge, NJ","Sep 19, 2013",my coworker brought me here one afternoon. it ...
636,Michelle S.,3,"New Milford, NJ","Jan 31, 2014","ive had better.....food is pretty good, staff ..."


used df.info to ensure no null values.

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 638 entries, 0 to 637
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   name      638 non-null    object
 1   rating    638 non-null    object
 2   location  638 non-null    object
 3   date      638 non-null    object
 4   body      638 non-null    object
dtypes: object(5)
memory usage: 25.0+ KB


applied a sentiment rating based on reviews of 3 rating or below as negative and all else (4,5) as positive.

In [51]:
df['rating'] = df['rating'].astype(int)

df['sentiment'] = df['rating'].apply(lambda x: 'Negative' if x <= 3 else 'Positive')

In [52]:
df

,name,rating,location,date,body,sentiment
0,Sarah P.,3,"Montclair, NJ","Mar 7, 2023",Let's start with the good - the ambiance and s...,Negative
1,Joseph F.,4,"New Milford, NJ","Dec 23, 2023",Pancho's food and service are very good BUT ri...,Positive
2,Dena H.,3,"Englewood, NJ","Jan 26, 2023",I really loved the vibes of this place and the...,Negative
3,Ellie O.,4,"Westwood, NJ","Jan 12, 2024",We have way too much fun when we come here! Fo...,Positive
4,Lenny G.,3,"Verona, NJ","Jan 14, 2023",The food is pretty mediocre and is way overpri...,Negative
...,...,...,...,...,...,...
633,Janis H.,2,"New Milford, NJ","Jun 16, 2010","This place is great for drinks, guac and chips...",Negative
634,Rita S.,5,"Manhattan, NY","Mar 4, 2012",The bean and cheese burritos are the best I've...,Positive
635,C B.,5,"Wood-Ridge, NJ","Sep 19, 2013",my coworker brought me here one afternoon. it ...,Positive
636,Michelle S.,3,"New Milford, NJ","Jan 31, 2014","ive had better.....food is pretty good, staff ...",Negative


In [53]:
file_name = restaurant_name + '.csv'
df.to_csv(file_name, index=False)

In [54]:
import nltk
from nltk.corpus import stopwords
import string
from sklearn.model_selection import train_test_split
from nltk.classify import NaiveBayesClassifier

nltk.download('stopwords')

def clean_text(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', string.punctuation))
    words = text.split()

    stop_words = set(stopwords.words('english'))
    words = [word for word in words if word not in stop_words]

    return words

cleaned_text = df['body'].apply(clean_text)

all_words = nltk.FreqDist(w.lower() for review in cleaned_text for w in review)
word_features = list(all_words.keys())

print(cleaned_text)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


0      [lets, start, good, ambiance, service, place, ...
1      [panchos, food, service, good, ridiculously, e...
2      [really, loved, vibes, place, music, also, day...
3      [way, much, fun, come, food, delicious, drinks...
4      [food, pretty, mediocre, way, overpriced, ambi...
                             ...                        
633    [place, great, drinks, guac, chips, maybe, avo...
634    [bean, cheese, burritos, best, ive, big, mexic...
635    [coworker, brought, one, afternoon, doesnt, lo...
636    [ive, betterfood, pretty, good, staff, iffy, w...
637    [family, always, loyal, customer, panchos, bur...
Name: body, Length: 638, dtype: object


Used the below code to create the bag of words and convert the words into a number of if theyre present in the review. Also converted the sentiment to positve =1 and negative =0.

In [61]:
from sklearn.feature_extraction.text import CountVectorizer

cleaned_text_sentences = cleaned_text.apply(lambda x: ' '.join(x))

vectorizer = CountVectorizer()

vectorizer.fit(cleaned_text_sentences)

X = vectorizer.transform(cleaned_text_sentences)

y = (df['sentiment'] == 'Positive').astype(int)



In [62]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)


used the below code to create the models and make predictions on the test set and separated each model.

In [63]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, precision_score, recall_score, roc_auc_score

naive_bayes = MultinomialNB()
decision_tree = DecisionTreeClassifier()
logistic_regression = LogisticRegression()

naive_bayes.fit(X_train, y_train)
decision_tree.fit(X_train, y_train)
logistic_regression.fit(X_train, y_train)

naive_bayes_preds = naive_bayes.predict(X_test)
decision_tree_preds = decision_tree.predict(X_test)
logistic_regression_preds = logistic_regression.predict(X_test)


In [64]:
from sklearn.metrics import classification_report, roc_auc_score

def evaluate_model(name, y_true, y_pred):
    print(f"{name} Classification Report:")
    print(classification_report(y_true, y_pred))

    roc_auc = roc_auc_score(y_true, y_pred)
    print(f"{name} ROC AUC Score: {roc_auc}")

evaluate_model("Naive Bayes", y_test, naive_bayes_preds)

evaluate_model("Decision Tree", y_test, decision_tree_preds)

evaluate_model("Logistic Regression", y_test, logistic_regression_preds)


Naive Bayes Classification Report:
              precision    recall  f1-score   support

           0       0.91      0.81      0.86       111
           1       0.77      0.89      0.83        81

    accuracy                           0.84       192
   macro avg       0.84      0.85      0.84       192
weighted avg       0.85      0.84      0.84       192

Naive Bayes ROC AUC Score: 0.8498498498498498
Decision Tree Classification Report:
              precision    recall  f1-score   support

           0       0.74      0.81      0.77       111
           1       0.70      0.60      0.65        81

    accuracy                           0.72       192
   macro avg       0.72      0.71      0.71       192
weighted avg       0.72      0.72      0.72       192

Decision Tree ROC AUC Score: 0.7078745412078745
Logistic Regression Classification Report:
              precision    recall  f1-score   support

           0       0.81      0.81      0.81       111
           1       0.74     

It seems as though the Naive Bayes is the best model all around. It has the highest ROCAUC by a large margin. The ROCAUC measures the ability to determine positive and negative cases making it the best way to determine positive and negative sentiment based on reviews.